## Import Libraries Needed

In [55]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.initializers import RandomNormal
from keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold

In [2]:
#declare random state:
rs = 1234

## Data Prep

### Load the Data in:

In [3]:
df = pd.read_csv("deeploc_dipeptide_percent_standard (1).csv")

In [4]:
df.head()

,seq_id,seq_description,AA,AC,AD,AE,AF,AG,AH,AI,...,YM,YN,YP,YQ,YR,YS,YT,YV,YW,YY
0,Q9H400,Q9H400_Cell.membrane-M_test,2.7211,0.3401,0.0000,0.6803,0.3401,1.7007,0.0000,0.3401,...,0.000,0.0000,0.0,0.3401,0.0000,0.6803,0.0000,0.0000,0.0000,0.000
1,Q5I0E9,Q5I0E9_Cell.membrane-M,0.7080,0.5310,0.1770,0.1770,0.3540,0.5310,0.1770,1.2389,...,0.177,0.0000,0.0,0.0000,0.0000,0.0000,0.1770,0.3540,0.0000,0.177
2,P63033,P63033_Cell.membrane-M,0.3774,0.0000,0.0000,0.7547,0.3774,0.3774,0.0000,0.0000,...,0.000,0.3774,0.0,0.3774,0.3774,0.0000,0.3774,0.3774,0.0000,0.000
3,Q9NR71,Q9NR71_Cell.membrane-M,0.5135,0.0000,0.2567,0.3851,0.3851,0.2567,0.2567,0.6418,...,0.000,0.0000,0.0,0.2567,0.3851,0.5135,0.2567,0.0000,0.1284,0.000
4,Q86XT9,Q86XT9_Cell.membrane-M,0.0000,0.0000,0.0000,0.0000,0.0000,1.6736,0.4184,0.0000,...,0.000,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000


In [5]:
df.shape

(14004, 402)

In [6]:
df.isnull().sum().max()

0

In [7]:
#Since we are working with NN's, we need to normalize the data: 

columns_to_drop = ["seq_id", "seq_description"]

from sklearn.preprocessing import normalize
df_norm = pd.DataFrame(normalize(df.drop(columns = columns_to_drop)), columns=df.drop(columns = columns_to_drop).columns.tolist())

In [8]:
df_norm.head()

,AA,AC,AD,AE,AF,AG,AH,AI,AK,AL,...,YM,YN,YP,YQ,YR,YS,YT,YV,YW,YY
0,0.272800,0.034096,0.000000,0.068203,0.034096,0.170501,0.000000,0.034096,0.000000,0.272800,...,0.000000,0.000000,0.0,0.034096,0.000000,0.068203,0.000000,0.000000,0.000000,0.000000
1,0.085497,0.064123,0.021374,0.021374,0.042748,0.064123,0.021374,0.149607,0.042748,0.256478,...,0.021374,0.000000,0.0,0.000000,0.000000,0.000000,0.021374,0.042748,0.000000,0.021374
2,0.045042,0.000000,0.000000,0.090072,0.045042,0.045042,0.000000,0.000000,0.180143,0.045042,...,0.000000,0.045042,0.0,0.045042,0.045042,0.000000,0.045042,0.045042,0.000000,0.000000
3,0.074040,0.000000,0.037013,0.055527,0.055527,0.037013,0.037013,0.092539,0.092539,0.037013,...,0.000000,0.000000,0.0,0.037013,0.055527,0.074040,0.037013,0.000000,0.018514,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.156530,0.039133,0.000000,0.000000,0.039133,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [9]:
df_norm.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
AA,14004.0,0.081855,0.083021,0.0,0.025412,0.063983,0.114977,0.955433
AC,14004.0,0.014716,0.025482,0.0,0.000000,0.000000,0.024609,0.394771
AD,14004.0,0.035113,0.036218,0.0,0.000000,0.029664,0.055021,0.304997
AE,14004.0,0.049359,0.046410,0.0,0.000000,0.042681,0.074655,0.487948
AF,14004.0,0.032380,0.035482,0.0,0.000000,0.026113,0.051375,0.286037
...,...,...,...,...,...,...,...,...
YS,14004.0,0.027726,0.032779,0.0,0.000000,0.021092,0.044797,0.392448
YT,14004.0,0.019077,0.027393,0.0,0.000000,0.000000,0.031830,0.237354
YV,14004.0,0.020717,0.027831,0.0,0.000000,0.000000,0.034443,0.294174
YW,14004.0,0.004955,0.013762,0.0,0.000000,0.000000,0.000000,0.203073


In [10]:
#add back the columns we dropped earlier:

df_norm["seq_id"] = df["seq_id"]
df_norm["seq_description"] = df["seq_description"]

In [11]:
df_norm.head()

,AA,AC,AD,AE,AF,AG,AH,AI,AK,AL,...,YP,YQ,YR,YS,YT,YV,YW,YY,seq_id,seq_description
0,0.272800,0.034096,0.000000,0.068203,0.034096,0.170501,0.000000,0.034096,0.000000,0.272800,...,0.0,0.034096,0.000000,0.068203,0.000000,0.000000,0.000000,0.000000,Q9H400,Q9H400_Cell.membrane-M_test
1,0.085497,0.064123,0.021374,0.021374,0.042748,0.064123,0.021374,0.149607,0.042748,0.256478,...,0.0,0.000000,0.000000,0.000000,0.021374,0.042748,0.000000,0.021374,Q5I0E9,Q5I0E9_Cell.membrane-M
2,0.045042,0.000000,0.000000,0.090072,0.045042,0.045042,0.000000,0.000000,0.180143,0.045042,...,0.0,0.045042,0.045042,0.000000,0.045042,0.045042,0.000000,0.000000,P63033,P63033_Cell.membrane-M
3,0.074040,0.000000,0.037013,0.055527,0.055527,0.037013,0.037013,0.092539,0.092539,0.037013,...,0.0,0.037013,0.055527,0.074040,0.037013,0.000000,0.018514,0.000000,Q9NR71,Q9NR71_Cell.membrane-M
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.156530,0.039133,0.000000,0.000000,0.039133,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Q86XT9,Q86XT9_Cell.membrane-M


In [12]:
df.head()

,seq_id,seq_description,AA,AC,AD,AE,AF,AG,AH,AI,...,YM,YN,YP,YQ,YR,YS,YT,YV,YW,YY
0,Q9H400,Q9H400_Cell.membrane-M_test,2.7211,0.3401,0.0000,0.6803,0.3401,1.7007,0.0000,0.3401,...,0.000,0.0000,0.0,0.3401,0.0000,0.6803,0.0000,0.0000,0.0000,0.000
1,Q5I0E9,Q5I0E9_Cell.membrane-M,0.7080,0.5310,0.1770,0.1770,0.3540,0.5310,0.1770,1.2389,...,0.177,0.0000,0.0,0.0000,0.0000,0.0000,0.1770,0.3540,0.0000,0.177
2,P63033,P63033_Cell.membrane-M,0.3774,0.0000,0.0000,0.7547,0.3774,0.3774,0.0000,0.0000,...,0.000,0.3774,0.0,0.3774,0.3774,0.0000,0.3774,0.3774,0.0000,0.000
3,Q9NR71,Q9NR71_Cell.membrane-M,0.5135,0.0000,0.2567,0.3851,0.3851,0.2567,0.2567,0.6418,...,0.000,0.0000,0.0,0.2567,0.3851,0.5135,0.2567,0.0000,0.1284,0.000
4,Q86XT9,Q86XT9_Cell.membrane-M,0.0000,0.0000,0.0000,0.0000,0.0000,1.6736,0.4184,0.0000,...,0.000,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000


### Obtain Y ground Truths from dataset:

In [14]:
df_norm["Cells"] = df_norm["seq_description"].str.split("_")

In [15]:
df_norm["Cell_Location"] = df_norm["Cells"].str[1]

In [16]:
df_norm["Cell_Location"].value_counts()

Nucleus-U                  3787
Cytoplasm-S                2542
Extracellular-S            1973
Cell.membrane-M            1340
Mitochondrion-U             761
Endoplasmic.reticulum-M     720
Mitochondrion-M             516
Plastid-U                   340
Golgi.apparatus-M           301
Plastid-M                   263
Lysosome/Vacuole-M          243
Mitochondrion-S             233
Plastid-S                   154
Cytoplasm-Nucleus-U         146
Nucleus-S                   134
Nucleus-M                   122
Peroxisome-U                100
Endoplasmic.reticulum-U      81
Lysosome/Vacuole-U           74
Endoplasmic.reticulum-S      61
Golgi.apparatus-U            53
Peroxisome-M                 48
Peroxisome-S                  6
Lysosome/Vacuole-S            4
Golgi.apparatus-S             2
Name: Cell_Location, dtype: int64

In [17]:
#based on the above info, remap these values into their unique cell location parts:

mapping_dict = {
    "Nucleus-U" : "Nucleus", 
    "Cytoplasm-S" : "Cytoplasm",              
    "Extracellular-S" : "Extracellular",         
    "Cell.membrane-M" : "Cell Membrane",        
    "Mitochondrion-U" : "Mitochondrion",             
    "Endoplasmic.reticulum-M" : "Endoplasmic Reticulum",   
    "Mitochondrion-M" : "Mitochondrion",         
    "Plastid-U" : "Plastid",                  
    "Golgi.apparatus-M" : "Golgi Apparatus",         
    "Plastid-M" : "Plastid",             
    "Lysosome/Vacuole-M" : "Lysosome/Vacuole",          
    "Mitochondrion-S" : "Mitochondrion", 
    "Plastid-S" : "Plastid",                 
    "Cytoplasm-Nucleus-U" : "Cytoplasm",        
    "Nucleus-S" : "Nucleus",                 
    "Nucleus-M" : "Nucleus",                 
    "Peroxisome-U" : "Peroxisome",             
    "Endoplasmic.reticulum-U" : "Endoplasmic Reticulum",   
    "Lysosome/Vacuole-U" : "Lysosome/Vacuole",          
    "Endoplasmic.reticulum-S" : "Endoplasmic Reticulum",     
    "Golgi.apparatus-U" : "Golgi Apparatus",         
    "Peroxisome-M" : "Peroxisome",                
    "Peroxisome-S" : "Peroxisome",                  
    "Lysosome/Vacuole-S" : "Lysosome/Vacuole",            
    "Golgi.apparatus-S" : "Golgi Apparatus"         
}

In [18]:
#Use a dictionary to create a new dictionary with the mapped values from above
new_mapping_dict = {key: mapping_dict[key] for key in df_norm["Cell_Location"].unique()}

#Replace the values in the "Cell_Location" column using the new mapped dictionary from above:
df_norm["Cell_Location"] = df_norm["Cell_Location"].map(new_mapping_dict)


In [19]:
df_norm["Cell_Location"].value_counts()

Nucleus                  4043
Cytoplasm                2688
Extracellular            1973
Mitochondrion            1510
Cell Membrane            1340
Endoplasmic Reticulum     862
Plastid                   757
Golgi Apparatus           356
Lysosome/Vacuole          321
Peroxisome                154
Name: Cell_Location, dtype: int64

In [20]:
df_norm.head()

,AA,AC,AD,AE,AF,AG,AH,AI,AK,AL,...,YR,YS,YT,YV,YW,YY,seq_id,seq_description,Cells,Cell_Location
0,0.272800,0.034096,0.000000,0.068203,0.034096,0.170501,0.000000,0.034096,0.000000,0.272800,...,0.000000,0.068203,0.000000,0.000000,0.000000,0.000000,Q9H400,Q9H400_Cell.membrane-M_test,"[Q9H400, Cell.membrane-M, test]",Cell Membrane
1,0.085497,0.064123,0.021374,0.021374,0.042748,0.064123,0.021374,0.149607,0.042748,0.256478,...,0.000000,0.000000,0.021374,0.042748,0.000000,0.021374,Q5I0E9,Q5I0E9_Cell.membrane-M,"[Q5I0E9, Cell.membrane-M]",Cell Membrane
2,0.045042,0.000000,0.000000,0.090072,0.045042,0.045042,0.000000,0.000000,0.180143,0.045042,...,0.045042,0.000000,0.045042,0.045042,0.000000,0.000000,P63033,P63033_Cell.membrane-M,"[P63033, Cell.membrane-M]",Cell Membrane
3,0.074040,0.000000,0.037013,0.055527,0.055527,0.037013,0.037013,0.092539,0.092539,0.037013,...,0.055527,0.074040,0.037013,0.000000,0.018514,0.000000,Q9NR71,Q9NR71_Cell.membrane-M,"[Q9NR71, Cell.membrane-M]",Cell Membrane
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.156530,0.039133,0.000000,0.000000,0.039133,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Q86XT9,Q86XT9_Cell.membrane-M,"[Q86XT9, Cell.membrane-M]",Cell Membrane


### Obtain Cell Cat Ordinal Nums:

In [21]:
categorical_var = {
    "Nucleus" : 0, 
    "Cytoplasm" : 1, 
    "Extracellular" : 2,
    "Mitochondrion" : 3,
    "Cell Membrane" : 4,
    "Endoplasmic Reticulum" : 5,
    "Plastid" : 6,
    "Golgi Apparatus" : 7,
    "Lysosome/Vacuole" : 8,
    "Peroxisome" : 9   
}

In [22]:
new_var_mapping_dict = {key: categorical_var[key] for key in df_norm["Cell_Location"].unique()}

df_norm["Cell_Cat"] = df_norm["Cell_Location"].map(new_var_mapping_dict)


In [23]:
df_norm.head()

,AA,AC,AD,AE,AF,AG,AH,AI,AK,AL,...,YS,YT,YV,YW,YY,seq_id,seq_description,Cells,Cell_Location,Cell_Cat
0,0.272800,0.034096,0.000000,0.068203,0.034096,0.170501,0.000000,0.034096,0.000000,0.272800,...,0.068203,0.000000,0.000000,0.000000,0.000000,Q9H400,Q9H400_Cell.membrane-M_test,"[Q9H400, Cell.membrane-M, test]",Cell Membrane,4
1,0.085497,0.064123,0.021374,0.021374,0.042748,0.064123,0.021374,0.149607,0.042748,0.256478,...,0.000000,0.021374,0.042748,0.000000,0.021374,Q5I0E9,Q5I0E9_Cell.membrane-M,"[Q5I0E9, Cell.membrane-M]",Cell Membrane,4
2,0.045042,0.000000,0.000000,0.090072,0.045042,0.045042,0.000000,0.000000,0.180143,0.045042,...,0.000000,0.045042,0.045042,0.000000,0.000000,P63033,P63033_Cell.membrane-M,"[P63033, Cell.membrane-M]",Cell Membrane,4
3,0.074040,0.000000,0.037013,0.055527,0.055527,0.037013,0.037013,0.092539,0.092539,0.037013,...,0.074040,0.037013,0.000000,0.018514,0.000000,Q9NR71,Q9NR71_Cell.membrane-M,"[Q9NR71, Cell.membrane-M]",Cell Membrane,4
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.156530,0.039133,0.000000,0.000000,0.039133,...,0.000000,0.000000,0.000000,0.000000,0.000000,Q86XT9,Q86XT9_Cell.membrane-M,"[Q86XT9, Cell.membrane-M]",Cell Membrane,4


### Train/Test Split:

In [27]:
columns_not_included = ["seq_id", "seq_description", "Cells", "Cell_Location", "Cell_Cat"]

In [28]:
X = df_norm.drop(columns = columns_not_included, axis=1)
y = df_norm["Cell_Cat"]

In [29]:
#Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = rs)

In [30]:
#we need to use binary values for y_values for the model: 

y_train_binaries= pd.get_dummies(y_train)
y_test_binaries= pd.get_dummies(y_test)

In [31]:
y_train_binaries.head()

,0,1,2,3,4,5,6,7,8,9
14,0,0,0,0,1,0,0,0,0,0
10004,1,0,0,0,0,0,0,0,0,0
12994,0,0,1,0,0,0,0,0,0,0
13874,0,0,1,0,0,0,0,0,0,0
10805,1,0,0,0,0,0,0,0,0,0


In [26]:
#when we go to create a confusion matrix, I'll have to hot encode these values back to Cell_Cat Values

## Q1. Train and test a fully connected feedforward NN with three layers:
- Input layer 400,
- Hidden layer 200 (ReLU)
- Output layer 10 (softmax) 

### Single Model Test (to start w/before implmenting CV function w/confusion matrices)

In [33]:
X_train.shape

(11203, 400)

In [34]:
#first create the model architecture:

model = keras.Sequential([
    layers.Dense(400, input_shape=(X_train.shape[1],)),
    layers.Dense(200, activation='relu'),
    layers.Dense(10, activation='softmax')
])

#Show the model summary:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 400)               160400    
                                                                 
 dense_1 (Dense)             (None, 200)               80200     
                                                                 
 dense_2 (Dense)             (None, 10)                2010      
                                                                 
Total params: 242,610
Trainable params: 242,610
Non-trainable params: 0
_________________________________________________________________


2023-05-10 14:50:42.979005: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [35]:
#we then need to compile the model:
model.compile(loss = "categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [36]:
#from textbook: then fit the model
history = model.fit(X_train, y_train_binaries, epochs=30, validation_data=(X_test, y_test_binaries))

Epoch 1/30
351/351 [==============================] - 4s 9ms/step - loss: 1.5332 - accuracy: 0.4563 - val_loss: 1.3326 - val_accuracy: 0.5198
Epoch 2/30
351/351 [==============================] - 2s 7ms/step - loss: 1.2742 - accuracy: 0.5507 - val_loss: 1.2844 - val_accuracy: 0.5423
Epoch 3/30
351/351 [==============================] - 3s 8ms/step - loss: 1.2011 - accuracy: 0.5798 - val_loss: 1.2885 - val_accuracy: 0.5419
Epoch 4/30
351/351 [==============================] - 3s 9ms/step - loss: 1.1416 - accuracy: 0.6005 - val_loss: 1.2712 - val_accuracy: 0.5498
Epoch 5/30
351/351 [==============================] - 2s 7ms/step - loss: 1.0862 - accuracy: 0.6216 - val_loss: 1.2540 - val_accuracy: 0.5516
Epoch 6/30
351/351 [==============================] - 2s 6ms/step - loss: 1.0391 - accuracy: 0.6417 - val_loss: 1.2573 - val_accuracy: 0.5494
Epoch 7/30
351/351 [==============================] - 2s 7ms/step - loss: 0.9897 - accuracy: 0.6565 - val_loss: 1.2261 - val_accuracy: 0.5598
Epoch 

In [38]:
#model train accuracy: 

loss,accuracy= model.evaluate(X_train,y_train_binaries)
print("The model loss is {:.2f}, its accuracy is {:.2f}. ".format(loss,accuracy))

351/351 [==============================] - 1s 3ms/step - loss: 0.1218 - accuracy: 0.9716
The model loss is 0.12, its accuracy is 0.97. 


In [39]:
#model test accuracy:

loss,accuracy= model.evaluate(X_test,y_test_binaries)
print("The model loss is {:.2f}, its accuracy is {:.2f}. ".format(loss,accuracy))

88/88 [==============================] - 0s 3ms/step - loss: 2.2942 - accuracy: 0.5469
The model loss is 2.29, its accuracy is 0.55. 


In [42]:
#we need to get y_pred values in order to create confusion matrix, but NN model produces probs of classes:


#we need to use the fit/trained model to predict X_test values and compare to ground truths: 
predicted = model.predict(X_test)

88/88 [==============================] - 0s 3ms/step


In [43]:
predicted[0]

array([9.8501539e-01, 3.7699981e-04, 3.9937622e-06, 1.0289195e-02,
       3.0024884e-07, 4.3121465e-03, 6.4147371e-10, 1.8127581e-07,
       1.7513083e-06, 7.8909385e-10], dtype=float32)

In [44]:
#we can implement the use of np.argmax() to return the instance with the highest probability:

np.argmax(predicted[0])

0

In [45]:
#after making the predictions, we then need to interpret probabilities:
y_predicted_labels = np.argmax(predicted, axis=1)

In [46]:
np.unique(y_predicted_labels)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [47]:
y_test.value_counts()

0    827
1    532
2    394
3    288
4    282
5    162
6    150
7     69
8     62
9     35
Name: Cell_Cat, dtype: int64

In [48]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predicted_labels)
cm

array([[518, 194,  23,  35,  24,   4,  10,  10,   7,   2],
       [138, 290,  14,  34,  14,  13,   6,   9,  12,   2],
       [ 14,  33, 269,  17,  20,   9,  14,   4,  11,   3],
       [ 38,  59,  14, 134,  10,  11,  14,   5,   0,   3],
       [ 17,  37,  11,  12, 143,  19,   8,   8,  26,   1],
       [ 17,  34,   1,  12,  18,  58,   5,   7,   9,   1],
       [ 13,  26,   3,  18,   2,   4,  77,   1,   2,   4],
       [  6,  17,   4,   5,   8,  13,   1,  11,   4,   0],
       [  3,   7,   2,   0,  19,   3,   3,   1,  24,   0],
       [  4,   6,   1,   6,   5,   1,   2,   1,   1,   8]])

With the help of referring back to our original cell_cat mapping values, we can interpret the order of the identifying column within the above matrices as follows: Nucleus, Cytoplasm, Extracellular, Mitochondrion, Cell Membrane, Endoplasmic Reticulum, Plastid, Golgi Apparatus, Lysosome/Vacuole, and Peroxisome. Based on this mapping, we can observe that for the Nucleus class, there were 518 correctly identified observations and 250 misclassified observations. Regarding the Cytoplasm, there were 290 observations correctly identified, while 413 observations were misclassified. This trend continues for other classes as well.

In [ ]:
#in order to interpret the above, refer back to the original cell_cat mapping values: 

# categorical_var = {
#     "Nucleus" : 0, 
#     "Cytoplasm" : 1, 
#     "Extracellular" : 2,
#     "Mitochondrion" : 3,
#     "Cell Membrane" : 4,
#     "Endoplasmic Reticulum" : 5,
#     "Plastid" : 6,
#     "Golgi Apparatus" : 7,
#     "Lysosome/Vacuole" : 8,
#     "Peroxisome" : 9   
# }

### Implementing K-Fold Cross Validation to Assess Model's Performance:

In [50]:
#unfortnately, we can't use this sklearn cross_val method, so instead, we'll have to create our own using KFold package:
# cross_val_score(model, X_train, y_train_binaries, cv=5)

In [51]:
#redefine X and y to start:

In [52]:
columns_not_included = ["seq_id", "seq_description", "Cells", "Cell_Location", "Cell_Cat"]

In [53]:
X = df_norm.drop(columns = columns_not_included, axis=1)
y = df_norm["Cell_Cat"]

In [54]:
#we then need to take 5 folds of data to implement 5 cv confusion matrices:

#first take the index locations of each respective fold and then run through it using a for-loop:

In [58]:
#Evaluate the performance from 5-fold cross validation using the confusion matrix as described in HW3. 

k = 5
# X_array = X.to_numpy()

#create Kfold
kf = KFold(n_splits=k, shuffle=True, random_state=rs)

#we need to get the indices of each *NEW* train/test from kfolds, which will then be used by the model within loop:
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    
    #After declaring the new train and test folds: train and test the model on it below:
    
    y_train_binaries= pd.get_dummies(y_train)
    y_test_binaries= pd.get_dummies(y_test)
    
    #create the model:
    model = keras.Sequential([
    layers.Dense(400, input_shape=(X_train.shape[1],)),
    layers.Dense(200, activation='relu'),
    layers.Dense(10, activation='softmax')])

    
    #compile the model:
    model.compile(loss = "categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    
    #train and fit the model --> only difference, set verbose = 0, so we don't see it printing out results: 
    history = model.fit(X_train, y_train_binaries, epochs=30, verbose=0, validation_data=(X_test, y_test_binaries))
    
    #make predictions using the model: 
    predicted = model.predict(X_test)

    #after making the predictions, we then need to interpret probabilities:
    y_predicted_labels = np.argmax(predicted, axis=1)
    
    #create confusion matrix for test set:
    cm = confusion_matrix(y_test, y_predicted_labels)
    print(cm)
    
    #print test accuracy: 
    loss,accuracy= model.evaluate(X_test,y_test_binaries)
    print("The model loss is {:.2f}, its accuracy is {:.2f}. ".format(loss,accuracy))

88/88 [==============================] - 0s 3ms/step
[[581 138  23  31  16   7  13   9   8   1]
 [160 240  28  36  14  18  13  10   7   6]
 [ 17  23 286  19  23   9  10   3   2   2]
 [ 40  55  11 135   9   6  18   7   1   6]
 [ 30  19  16  10 158  14   7  11  15   2]
 [ 20  28   4  11  25  49   4   8  11   2]
 [ 10  16   3  18   6   4  86   1   1   5]
 [  8  17   6   7   6   8   1  13   3   0]
 [  2  10   3   1  17   2   1   6  20   0]
 [  5   6   2   4   4   2   2   1   0   9]]
88/88 [==============================] - 0s 3ms/step - loss: 2.2399 - accuracy: 0.5630
The model loss is 2.24, its accuracy is 0.56. 
88/88 [==============================] - 0s 2ms/step
[[539 188  23  34  15  11  12   4   0   1]
 [142 289  25  29  15   9  13  16   2   0]
 [ 20  17 302  11  13  12   7   4   1   0]
 [ 37  59  13 148   9  12  27   6   2   1]
 [ 18  30  12   7 137  24   3  10   7   1]
 [ 17  24   9  12  28  57   3  13   4   1]
 [ 13  23   4  26   4   7  72   5   2   0]
 [  9   8   1   4  13   8   

From the above kfold cross validation matrices produced, we can see that the model has an average accuracy of 56%. Delving deeper into assessing the models performance, we can see that the model has similar classifying capabilities when being trained/tested on different folds of data. Even though our accuracy levels might not be the highest, this does support the notion that our model is able to generalize well when given different training and testing partitions of the dataset.

In [ ]:
#in the future I should attempt to implement a stratified grouped kfold --> look at other CV methods

## Q2. Train and test a fully connected feedforward DNN with 6 layers: 400, 200, 100, 50, 25, 10.
- a) Use sigmoid activation function, and random initialization using a normal distribution with a mean of 0 and a standard deviation of 1.
- b) Use LeakyLU activation function, HE initialization, and Adam Optimization 

Discuss and compare performance of the various settings. 

Evaluate the performance from 5-fold cross validation using the confusion matrix as described
in HW3. 

### A) Use sigmoid activation function, and random initialization using a normal distribution with a mean of 0 and a standard deviation of 1.

In [ ]:
#new model architecture w/sigmoid activation function and Random Normal initializer using mean=0 ans stddev = 1:

model = Sequential()
model.add(Dense(400, activation='sigmoid', input_shape=(X_train.shape[1],), kernel_initializer=RandomNormal(mean=0, stddev=1)))
model.add(Dense(200, activation='sigmoid'))
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(50, activation='sigmoid'))
model.add(Dense(25, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))


# Print the model summary
model.summary()


In [59]:
#Evaluate the performance from 5-fold cross validation using the confusion matrix as described in HW3. 

k = 5
# X_array = X.to_numpy()

#create Kfold
kf = KFold(n_splits=k, shuffle=True, random_state=rs)

#we need to get the indices of each *NEW* train/test from kfolds, which will then be used by the model within loop:
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    
    #After declaring the new train and test folds: train and test the model on it below:
    
    y_train_binaries= pd.get_dummies(y_train)
    y_test_binaries= pd.get_dummies(y_test)
    
    #Change the model: new model architecture w/sigmoid activation function and Random Normal initializer using mean=0 ans stddev = 1:
    
    model = Sequential()
    model.add(Dense(400, activation='sigmoid', input_shape=(X_train.shape[1],), kernel_initializer=RandomNormal(mean=0, stddev=1)))
    model.add(Dense(200, activation='sigmoid'))
    model.add(Dense(100, activation='sigmoid'))
    model.add(Dense(50, activation='sigmoid'))
    model.add(Dense(25, activation='sigmoid'))
    model.add(Dense(10, activation='sigmoid'))

    
    #compile the model:
    model.compile(loss = "categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    
    #train and fit the model --> only difference, set verbose = 0, so we don't see it printing out results: 
    history = model.fit(X_train, y_train_binaries, epochs=30, verbose=0, validation_data=(X_test, y_test_binaries))
    
    #make predictions using the model: 
    predicted = model.predict(X_test)

    #after making the predictions, we then need to interpret probabilities:
    y_predicted_labels = np.argmax(predicted, axis=1)
    
    #create confusion matrix for test set:
    cm = confusion_matrix(y_test, y_predicted_labels)
    print(cm)
    
    #print test accuracy: 
    loss,accuracy= model.evaluate(X_test,y_test_binaries)
    print("The model loss is {:.2f}, its accuracy is {:.2f}. ".format(loss,accuracy))

88/88 [==============================] - 0s 3ms/step
[[705  74  18  17   7   2   4   0   0   0]
 [293 160  25  39   5   3   7   0   0   0]
 [ 24  21 302  26  13   6   2   0   0   0]
 [ 76  50  21 120   5   3  13   0   0   0]
 [ 51  17  26   3 163  13   9   0   0   0]
 [ 18  34   9  10  41  43   7   0   0   0]
 [ 29  22   3  31   5   4  56   0   0   0]
 [ 29  11   6   3  11   9   0   0   0   0]
 [  6  10   8   1  32   5   0   0   0   0]
 [  5   5   4   9   6   1   5   0   0   0]]
88/88 [==============================] - 0s 3ms/step - loss: 1.3621 - accuracy: 0.5530
The model loss is 1.36, its accuracy is 0.55. 
88/88 [==============================] - 0s 3ms/step
[[584 145  23  44  21   4   6   0   0   0]
 [198 210  33  61  27   7   4   0   0   0]
 [ 15  19 310  22  15   6   0   0   0   0]
 [ 39  49  23 161   8  13  21   0   0   0]
 [ 27  16  17   5 161  21   2   0   0   0]
 [ 13  28  16   7  49  49   6   0   0   0]
 [ 12  27   3  46   8   4  56   0   0   0]
 [  3  19   3   7  20  12   

In [1]:
#in order to interpret the above, refer back to the original cell_cat mapping values: 

# categorical_var = {
#     "Nucleus" : 0, 
#     "Cytoplasm" : 1, 
#     "Extracellular" : 2,
#     "Mitochondrion" : 3,
#     "Cell Membrane" : 4,
#     "Endoplasmic Reticulum" : 5,
#     "Plastid" : 6,
#     "Golgi Apparatus" : 7,
#     "Lysosome/Vacuole" : 8,
#     "Peroxisome" : 9   
# }

Within the above confusion matrices, we can see that this particular neural network's architecture struggles to correctly classify certain class instances. This model continuously misclassifies the Golgi Apparatus, Lysosome/Vacuole, and Peroxisome. Overall, these results suggest that the model struggles with certain classes while performing relatively better in classifying the Nucleus class. The accuracy levels achieved by this neural network model range from values around 53% to 56%. Furthermore, we can observe this model's moderate performance reflected in the training phase, as the neural network's objective loss function (errors) is minimized.

### B) Use LeakyLU activation function, HE initialization, and Adam Optimization

In [ ]:
#from the textbook page 332: 

# leaky_relu = keras.layers.LeakyReLU(alpha=0.2)
#     layer = keras.layers.Dense(10, activation=leaky_relu,
#                                kernel_initializer="he_normal")

In [ ]:
#look at page 329 in textbook for info about the HE initialization function: 
#look at page 332 in textbook for info about how to create LeakyLU functions:

leaky_relu = keras.layers.LeakyReLU(alpha=0.2)

model = Sequential()
model.add(Dense(400, activation = leaky_relu, input_shape = (X_train.shape[1],), kernel_initializer="he_normal"))
model.add(Dense(200, activation=leaky_relu))
model.add(Dense(100, activation=leaky_relu))
model.add(Dense(50, activation=leaky_relu))
model.add(Dense(25, activation=leaky_relu))
model.add(Dense(10, activation=leaky_relu))



#Print the model summary
model.summary()


In [60]:
#Evaluate the performance from 5-fold cross validation using the confusion matrix as described in HW3. 

k = 5
# X_array = X.to_numpy()

#create Kfold
kf = KFold(n_splits=k, shuffle=True, random_state=rs)

#we need to get the indices of each *NEW* train/test from kfolds, which will then be used by the model within loop:
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    
    #After declaring the new train and test folds: train and test the model on it below:
    
    y_train_binaries= pd.get_dummies(y_train)
    y_test_binaries= pd.get_dummies(y_test)
    
    #Change the model: new model architecture w/LeakyLU activation function, HE initialization, and Adam Optimization:
    
    leaky_relu = keras.layers.LeakyReLU(alpha=0.2)

    model = Sequential()
    model.add(Dense(400, activation = leaky_relu, input_shape = (X_train.shape[1],), kernel_initializer="he_normal"))
    model.add(Dense(200, activation=leaky_relu))
    model.add(Dense(100, activation=leaky_relu))
    model.add(Dense(50, activation=leaky_relu))
    model.add(Dense(25, activation=leaky_relu))
    model.add(Dense(10, activation=leaky_relu))

    
    #compile the model:
    model.compile(loss = "categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    
    #train and fit the model --> only difference, set verbose = 0, so we don't see it printing out results: 
    history = model.fit(X_train, y_train_binaries, epochs=30, verbose=0, validation_data=(X_test, y_test_binaries))
    
    #make predictions using the model: 
    predicted = model.predict(X_test)

    #after making the predictions, we then need to interpret probabilities:
    y_predicted_labels = np.argmax(predicted, axis=1)
    
    #create confusion matrix for test set:
    cm = confusion_matrix(y_test, y_predicted_labels)
    print(cm)
    
    #print test accuracy: 
    loss,accuracy= model.evaluate(X_test,y_test_binaries)
    print("The model loss is {:.2f}, its accuracy is {:.2f}. ".format(loss,accuracy))

88/88 [==============================] - 0s 2ms/step
[[622   0 194   0  11   0   0   0   0   0]
 [405   0 110   0  17   0   0   0   0   0]
 [165   0 199   0  30   0   0   0   0   0]
 [219   0  47   0  22   0   0   0   0   0]
 [186   0  34   0  62   0   0   0   0   0]
 [112   0  22   0  28   0   0   0   0   0]
 [100   0  36   0  14   0   0   0   0   0]
 [ 55   0   8   0   6   0   0   0   0   0]
 [ 36   0   4   0  22   0   0   0   0   0]
 [ 26   0   3   0   6   0   0   0   0   0]]
88/88 [==============================] - 0s 2ms/step - loss: 7.4809 - accuracy: 0.3152
The model loss is 7.48, its accuracy is 0.32. 
88/88 [==============================] - 0s 2ms/step
[[  0 587   0 121   0   0   0   0 119   0]
 [  0 325   0  96   0   0   0   0 119   0]
 [  0  97   0 241   0   0   0   0  49   0]
 [  0 169   0  59   0   0   0   0  86   0]
 [  0  91   0  41   0   0   0   0 117   0]
 [  0  67   0  40   0   0   0   0  61   0]
 [  0 103   0  17   0   0   0   0  36   0]
 [  0  27   0   9   0   0   

In [2]:
#in order to interpret the above, refer back to the original cell_cat mapping values: 

# categorical_var = {
#     "Nucleus" : 0, 
#     "Cytoplasm" : 1, 
#     "Extracellular" : 2,
#     "Mitochondrion" : 3,
#     "Cell Membrane" : 4,
#     "Endoplasmic Reticulum" : 5,
#     "Plastid" : 6,
#     "Golgi Apparatus" : 7,
#     "Lysosome/Vacuole" : 8,
#     "Peroxisome" : 9   
# }

Given the above output, we are able to see that this deep learning neural network struggles to accurately classify the ground truths. As a result of this, the model's average accuracy was only 17.2%. This low accuracy can most likely be attributed to the model's deep neural network being overfit. Within this model's architecture, there were 4 hidden layers. I suspect that as a result of this, and in addition to the low regularization methods employed, this model's overall performance is not as good as the others.